# Exploratory Data Analysis on a single SC2 Replay File

This notebook details the process for loading and exploring a single SC2 replay file. The following goals are achieved:
1. Load the replay file using `sc2reader`
2. Extract the events stored in the `.events` attribute of the replay file into a pandas dataframe.
3. Extract other pertinent information from the replay file such as: 
    * Filehash.
    * Datetime the game was played.
    * The map on which the game was played.
    * Length of the game.
    * Races of the players.
    * Skill levels of the players (by league).
    * The winner of the game.
4. Drop all unnecessary columns, convert the rest of the columns into dummy variables which indicate the category of the action and 1 if the action was taken.
5. Divide the data up into time chunks (10 seconds initially, to be revised later). Sum the number of 1s in each chunk to get the number of actions taken in each chunk.
6. Add columns to the dataframe indicating the number of actions taken in all previous chunks.
7. Add columns to the dataframe indicating player races, skill levels, and game winner.
8. Attempt a logistic regression using all chunks as inputs.
9. Investigate an optimization method which results in the earliest prediction that does not change until the end of the game. I.e., how early can the regression make a prediction which remains constant until the end of the game.

See __[this link](https://web.archive.org/web/20201031184319/https://miguelgondu.github.io/python/ai/video%20games/2018/09/04/a-tutorial-on-sc2reader-events-and-units.html)__ for the tutorial that was used in establishing how to work with, and extract data from an SC2 replay file, using `sc2reader`, and __[this link](https://github.com/IBM/starcraft2-replay-analysis)__ for an example of analyzing a single SC2 replay file using `sc2reader` and Jupyter Notebooks.

In [2]:
# imports and settings

import pandas as pd
import numpy as np
import sc2reader
from multiprocessing import Pool # for parallel processing of replay files
from scripts.classes import ReplayInfo
import re

## Load a Replay
We will use the `load_level=4` setting to extract the most detailed data possible from the replay file. This includes all events, units, and upgrades.

The replay selected is a professional replay between Serral (Zerg) and Showtime (Protoss). The replay is 42 minutes long (which is long for a StarCraft II game) which should be a good indication of the upper end of the processing time and requirements for a replay.

In [3]:
# Load a replay
replay_path = "data/replays/set_2/000034772b83aed5732879a679487bbd496304b1490313ae9daaef3117391707.SC2Replay"
replay = sc2reader.load_replay(replay_path, load_level=4)

if replay.game_type == '1v1':
    # Extract a list of replay events
    events_list = replay.events
else:
    print("NOTE - This replay has more than 2 players. Not valid!!!")

## Extract Event Data from Replay
The following code extracts the events from the replay file and stores them in a pandas dataframe. Multiproccessing is used to speed up the process, as the replay file contains over 150,000 events.

Steps:
1. `events_list` now contains a list of stored event objects. `get_event_data` is a function that takes a specific event, and returns a dictionary of the values for each attribute of that event object. 
2. A multiprocessing pool iterates through all events returning a dictionary for each.
3. The output from the pool is stored in a list of dictionaries which can then be compressed into a pandas dataframe. A list of dictionaries is used so that all column names for all events do not need to be created manually.

In [4]:
# create function to get the event data so that multiprocessing can be used
def get_event_data(event):
    """
    get_event_data
    Extracts the data from the event and returns it as a dictionary
    Ignores events that start with '_', i.e., special attributes and dunder types

    Args:
        event (sc2reader.event): Event object extracted from sc2reader.events

    Returns:
        [dict]: A dictionary containing the event data
    """    
    # ignore attributes that are not needed (special or dunder)
    event_attributes = [attr for attr in dir(event) if not attr.startswith('_')]

    # initialize a dictionary to store the values of each attribute
    d = dict()

    # loop through each attribute and store the value in the dictionary
    for attr in event_attributes:
        # ignore attributes if they do not contain a value type
        if type(getattr(event, attr)) in [int, float, str, bool]:
            d[attr] = getattr(event, attr)
    
    return d

In [5]:
# run the function on each event in the list, returns a list of dictionaries
# with Pool() as pool:
#     dict_list = pool.map(get_event_data, events_list)
dict_list = [get_event_data(event) for event in events_list]
# convert the list of dictionaries to a dataframe
event_df = pd.DataFrame(dict_list)
event_df.shape

(6946, 35)

In [45]:
# check all column names
for col in event_df.columns:
    print(col)

base_build_num
debug_pause_enabled
development_cheats_enabled
frame
game_fully_downloaded
is_local
is_map_to_map_transition
multiplayer_cheats_enabled
name
pid
second
sync_checksumming_enabled
x
y
bank
control_group
mask_type
subgroup_index
ability_id
ability_link
ability_name
ability_type
command_index
flags
has_ability
ability_data
z
hotkey
update_type
control_player_id
target_flags
target_timer
target_unit_id
target_unit_type
upkeep_player_id


We now have an `event_df` with 151,759 rows and 112 columns.

## Processing the Dataframe
The major steps to accomplish here are:
1. Identify all columns.
2. Drop all columns that are not needed. We will also store a list of columns that are left after dropping in a csv file in the info directory. This list can be used later as a master list of columns for processing of all dataframes.
3. Check if columns are continuous or categorical (discrete integers are considered categorical).
4. Convert all categorical columns into dummy variables and drop the original column, and the first dummy column (using the pandas `drop first` option).
5. Remove all rows with non-player PID values (the players are the first two PIDs in the replay, all other PIDs are observers).
6. Summarise the dataframe by dividing it up into chunks of 10 seconds, summing up the count of all events during that time.


In [5]:
# # print the names of the columns to a csv
# pd.DataFrame(event_df.columns).to_csv(
#     '../info/raw_columns_list.csv',
#     index=False)

In [22]:
# read the csv into columns_checklist
columns_checklist = pd.read_csv('info/raw_columns_list.csv')
columns_checklist

,column_name,include_bool,categorical_bool,agg_type
0,frame,1,0,none
1,name,1,1,sum
2,pid,1,1,none
3,second,1,0,diff
4,sid,0,0,none
...,...,...,...,...
107,target,0,0,none
108,text,0,0,none
109,to_all,0,0,none
110,to_allies,0,0,none


In [23]:
# create a dataframe of all columns with to be included
column_keep_df = columns_checklist.loc[
    columns_checklist['include_bool'] == 1].drop(['include_bool'], axis=1)
column_keep_df

,column_name,categorical_bool,agg_type
0,frame,0,none
1,name,1,sum
2,pid,1,none
3,second,0,diff
7,count,0,sum
8,upgrade_type_name,1,sum
13,unit_type_name,1,sum
26,ff_minerals_lost_army,0,sum
27,ff_minerals_lost_economy,0,sum
28,ff_minerals_lost_technology,0,sum


In [24]:
# We now have a list of 68 columns that we want to keep
# We can use this list to create a new dataframe with only the columns we want
event_df = event_df[column_keep_df['column_name']]
event_df.shape

(1890, 60)

### Create Masks for Categorical and Continuous Columns
We will create two masks one for all categorical columns and one for all continuous columns.

First we need to identify which columns are continuous and which columns are categorical.

In [25]:
# identify which columns are continuous and which are categorical
categorical_columns = column_keep_df.loc[
        column_keep_df['categorical_bool'] == 1, 'column_name'].tolist()
continuous_columns = column_keep_df.loc[
        column_keep_df['categorical_bool'] == 0, 'column_name'].tolist()

In [26]:
print('<<<<Categorical columns>>>>\n',categorical_columns)

<<<<Categorical columns>>>>
 ['name', 'pid', 'upgrade_type_name', 'unit_type_name', 'ability_id', 'killer_pid']


In [27]:
print('<<<<Continuous columns>>>>\n',continuous_columns)

<<<<Continuous columns>>>>
 ['frame', 'second', 'count', 'ff_minerals_lost_army', 'ff_minerals_lost_economy', 'ff_minerals_lost_technology', 'ff_vespene_lost_army', 'ff_vespene_lost_economy', 'ff_vespene_lost_technology', 'food_made', 'food_used', 'minerals_collection_rate', 'minerals_current', 'minerals_killed', 'minerals_killed_army', 'minerals_killed_economy', 'minerals_killed_technology', 'minerals_lost', 'minerals_lost_army', 'minerals_lost_economy', 'minerals_lost_technology', 'minerals_used_active_forces', 'minerals_used_current', 'minerals_used_current_army', 'minerals_used_current_economy', 'minerals_used_current_technology', 'minerals_used_in_progress', 'minerals_used_in_progress_army', 'minerals_used_in_progress_economy', 'minerals_used_in_progress_technology', 'resources_killed', 'resources_lost', 'resources_used_current', 'resources_used_in_progress', 'vespene_collection_rate', 'vespene_current', 'vespene_killed', 'vespene_killed_army', 'vespene_killed_economy', 'vespene_k

There are 13 categorical columns and 55 continuous columns. 

1 of the continuous columns is 'count', which appears to count the number of an event that happen during a frame. This column will be used to insert the values upon dummification of the categorical columns. In other words, when categories are converted to dummy variables the value inside a column, for a specific row, will be multiplied by the value of the count column. In general, count appears to remain at 1, but this is done just in case it is not true for every replay. After the multiplication, the count column should be dropped.

### Continuous Columns
The value of the columns in the continuous set should be investigated to determine the best way of aggregating the values. For example, mineral collection rate would need to be averaged over the time chunks (this is stored as a value of Minerals collected per Minute), whereas the number of units killed would need to be summed. It would be best to include an aggregation type column in the `column_keep_df` which could be used during aggregation. This value can be added to the raw_columns_list.csv to automate the process.

In [28]:
event_df[continuous_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
frame,1890.0,4096.274603,2989.079525,0.0,1224.5,4360.0,6814.00,9010.0
second,1890.0,255.665079,186.713170,0.0,76.0,272.5,425.75,563.0
count,17.0,1.000000,0.000000,1.0,1.0,1.0,1.00,1.0
ff_minerals_lost_army,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
ff_minerals_lost_economy,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
ff_minerals_lost_technology,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
ff_vespene_lost_army,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
ff_vespene_lost_economy,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
ff_vespene_lost_technology,115.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
food_made,115.0,41.478261,17.145583,15.0,27.0,47.0,55.00,78.0


### Categorical Columns

All categorical columns with an aggregation type specified will be dummified so that value is one or zero. During aggregation each time chunk will be aggregated by multiplying a columns value by count, and then summing a count of the occurrences within that chunk. 
Aggregation for categorical variables will follow these steps:
1. Dummy variables will be created for all necessary categorical columns.
2. Separate temporary dfs will be created for each players PID, and one for events with out PID
3. A helper column will be added to the temp dfs, calculated from frame number, to identify grouping of rows into time chunks (for 10 second frames, this would be chunks of `10*fps`).
4. Each column in the temp df will have pid added to its name if not NaN and then the dfs will be recombined to produced single df with 2 columns for every variable with a pid attached.
5. Continuous columns can be dropped from the df where PID = NaN as continuous variables only apply to PIDs
6. Perform aggregation on recombined df

In [29]:
event_df[categorical_columns].describe(include='all').T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,1890,22,CameraEvent,376,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pid,1346.0,NaN,NaN,NaN,0.145617,0.465509,0.0,0.0,0.0,0.0,2.0
upgrade_type_name,17,13,SprayTerran,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unit_type_name,450,65,InvisibleTargetDummy,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ability_id,271.0,NaN,NaN,NaN,2080.833948,3875.598438,0.0,0.0,0.0,4194.5,19776.0
killer_pid,53.0,NaN,NaN,NaN,1.113208,0.319878,1.0,1.0,1.0,1.0,2.0


#### 1. Dummy variable creation
Before creating Dummy variables it makes sense to remove all rows that do not contain information about the PIDs that we are interested in (pid 1 and 2). Rows that include data about game information will have a PID of NaN, so we will keep these too.

Initially, the first dummy column created will be dropped. This may be revised later.


In [30]:
# remove all rows where pid not NaN and pid = 1 or 2 

event_df = event_df.loc[~(
    (event_df['pid'].notnull()) 
    & (event_df['pid'] != 1) 
    & (event_df['pid'] != 2))]

event_df.shape

(678, 60)

In [31]:
event_df['pid'].isna().sum()

544

In [32]:
# list of column_name in column_keep_df with agg_type not equal to none or
agg_columns = column_keep_df.loc[
    (column_keep_df['agg_type'] != 'none') 
    & (column_keep_df['agg_type'] != 'diff')
    & (column_keep_df['categorical_bool'] == 1),
    'column_name'].tolist()

# create dummies from event_df for all columns in agg_columns
event_df = pd.get_dummies(
    event_df, 
    prefix=agg_columns,
    columns=agg_columns,
    drop_first=True)

event_df



,frame,pid,second,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,...,unit_type_name_SupplyDepotLowered,unit_type_name_TechLab,unit_type_name_UnbuildableBricksDestructible,unit_type_name_VespeneGeyser,unit_type_name_VikingAssault,unit_type_name_VikingFighter,unit_type_name_WarpGate,unit_type_name_XelNagaTower,unit_type_name_Zealot,killer_pid_2.0
0,0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,8905,NaN,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1885,8909,NaN,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1886,8936,2.0,558,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1887,8936,2.0,558,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


#### 2. Split into temporary dfs
3 temp dfs will be created, one for each PID and one for PID of NaN.

In [33]:
# df for PID = 1
df_1 = event_df.loc[
    event_df['pid'] == 1].copy()

# df for PID = 2
df_2 = event_df.loc[
    event_df['pid'] == 2].copy()

# df for PID is NaN
df_nan = event_df.loc[
    event_df['pid'].isnull()].copy()



#### 3. Add helper column to temp dfs
A helper column will be added that calculates which frame interval a row is in. This will be used with `groupby()` to aggregate the three dfs

In [34]:
# function to add helper columns for aggregation by second
def add_frame_aggregation_column(df, period, fps, new_column_name='agg_column'):
    """
    add_frame_aggregation_column 
    Adds a column to the dataframe that contains the second number at the start of the chunk. This second number will be used to aggregate the data.

    Args:
        df (DataFrame): DataFrame to add the column to
        period (int): The number of frames in a chunk
        fps (int): The number of frames per second
        new_column_name (str, optional): The name of the new column to add. Defaults to 'agg_column'.

    Returns:
        DataFrame: The dataframe with the new column added
    """    
    assert isinstance(df, pd.DataFrame), 'df must be a dataframe'

    # divide frame by period and round down to integer
    # then multiply by period / fps to get the start second of the chunk
    df[new_column_name] = (df['frame'] // period) * period / fps

    return df



In [35]:
fps = replay.game_fps
seconds_per_chunk = 10
period = int(fps * seconds_per_chunk)

df_1 = add_frame_aggregation_column(
                    df_1, period, fps, 'start_second').copy()
df_2 = add_frame_aggregation_column(
                    df_2, period, fps, 'start_second').copy()
df_nan = add_frame_aggregation_column(
                    df_nan, period, fps, 'start_second').copy()

In [36]:
# add pid identifier to all columns in df_1 and df_2
# drop frame, second, pid
def drop_my_columns(df, drop_columns=['frame', 'second', 'pid']):
    # drop columns inplace
    df.drop(drop_columns, axis=1, inplace=True)

    return df

def add_pid(df, pid):
    # rename all columns by adding 'pid_' to the front
    df.columns = ['pid_' + str(pid) + '_' + col for col in df.columns]

    return df


In [37]:
# loop through dfs and drop columns, 
# for i, df in enumerate([df_1, df_2]):
#     df = drop_my_columns(df).copy()
#     df.set_index('start_second', inplace=True)
#     df = add_pid(df, i + 1).copy()

df_1 = drop_my_columns(df_1).copy()
df_1.set_index('start_second', inplace=True)
df_1 = add_pid(df_1, 1).copy()

df_2 = drop_my_columns(df_2).copy()
df_2.set_index('start_second', inplace=True)
df_2 = add_pid(df_2, 2).copy()


In [38]:
df_nan

,frame,pid,second,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,...,unit_type_name_TechLab,unit_type_name_UnbuildableBricksDestructible,unit_type_name_VespeneGeyser,unit_type_name_VikingAssault,unit_type_name_VikingFighter,unit_type_name_WarpGate,unit_type_name_XelNagaTower,unit_type_name_Zealot,killer_pid_2.0,start_second
11,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
12,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
13,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
15,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,8887,NaN,555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,550.0
1882,8890,NaN,555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,550.0
1883,8897,NaN,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,550.0
1884,8905,NaN,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,550.0


In [39]:
# drop continuous_columns from df_nan
columns_to_drop = continuous_columns[:] + ['pid']
df_nan.drop(columns_to_drop, axis=1, inplace=True)
df_nan.set_index('start_second', inplace=True)

# drop all columns except those with sum > 0
df_nan = df_nan.loc[:,df_nan.sum(axis=0) > 0].copy()


In [40]:
df_1.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
pid_1_count,14.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
pid_1_ff_minerals_lost_army,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_ff_minerals_lost_economy,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_ff_minerals_lost_technology,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_ff_vespene_lost_army,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
pid_1_unit_type_name_VikingFighter,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_unit_type_name_WarpGate,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_unit_type_name_XelNagaTower,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_1_unit_type_name_Zealot,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_2.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
pid_2_count,3.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
pid_2_ff_minerals_lost_army,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_ff_minerals_lost_economy,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_ff_minerals_lost_technology,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_ff_vespene_lost_army,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
pid_2_unit_type_name_VikingFighter,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_unit_type_name_WarpGate,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_unit_type_name_XelNagaTower,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pid_2_unit_type_name_Zealot,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_nan.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
name_UnitBornEvent,544.0,0.685662,0.464679,0.0,0.0,1.0,1.0,1.0
name_UnitDiedEvent,544.0,0.097426,0.296811,0.0,0.0,0.0,0.0,1.0
name_UnitDoneEvent,544.0,0.064338,0.245580,0.0,0.0,0.0,0.0,1.0
name_UnitInitEvent,544.0,0.071691,0.258213,0.0,0.0,0.0,0.0,1.0
name_UnitPositionsEvent,544.0,0.011029,0.104536,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
unit_type_name_VikingFighter,544.0,0.009191,0.095517,0.0,0.0,0.0,0.0,1.0
unit_type_name_WarpGate,544.0,0.003676,0.060578,0.0,0.0,0.0,0.0,1.0
unit_type_name_XelNagaTower,544.0,0.001838,0.042875,0.0,0.0,0.0,0.0,1.0
unit_type_name_Zealot,544.0,0.012868,0.112807,0.0,0.0,0.0,0.0,1.0
